In [ ]:
# paral
import requests
import pandas as pd
from IPython.display import display
import websockets
import asyncio
import json
import talib as ta
symbol='BTCUSDT'
interval= '1d'

API_KEY = "368115acbe78e27f4c7dc324b142afb0756f17045ce46fdd3ad030857b4efa91"
API_SECRET = "1f0ad0bb68ac295ffb3c6e1a59e16b25e63a1cb80dbec27f9b6a294c9376e66f"

url=f'https://api.binance.com/api/v3/klines?symbol={symbol}&interval={interval}&limit=5000'
data = requests.get(url=url)
data = data.json()


class PlaceOrder:
    def __init__(self):
        self.API_KEY = "368115acbe78e27f4c7dc324b142afb0756f17045ce46fdd3ad030857b4efa91"
        self.API_SECRET = "1f0ad0bb68ac295ffb3c6e1a59e16b25e63a1cb80dbec27f9b6a294c9376e66f"
        self.BASE_URL = 'https://testnet.binance.vision'
        self.endpoint = '/api/v3/order'
    def buy(self):
        
        response = requests.get(BASE_URL + '/api/v3/time')
        server_time = response.json()['serverTime']
        self.params = {
        'symbol': 'BTCUSDT',
        'side': 'BUY',
        'type': 'MARKET',
        'quantity': 0.001,
        'recvWindow': 5000,  # Time for the request to be valid in milliseconds
        'timestamp': server_time, 
            }



        selfparams['signature'] = self.create_signature(self.params, self.API_SECRET)


        headers = {
        'X-MBX-APIKEY': self.API_KEY
        }

        response = requests.post(self.BASE_URL + self.endpoint, headers=headers, params=self.params)
        print(response.json())
        

    def sell(self):
        response = requests.get(BASE_URL + '/api/v3/time')
        server_time = response.json()['serverTime']
        self.params = {
        'symbol': 'BTCUSDT',
        'side': 'SELL',
        'type': 'MARKET',
        'quantity': 0.001,
        'recvWindow': 5000,  # Time for the request to be valid in milliseconds
        'timestamp': server_time, 
            }

        selfparams['signature'] = self.create_signature(self.params, self.API_SECRET)
        headers = {
        'X-MBX-APIKEY': self.API_KEY
        }

        response = requests.post(self.BASE_URL + self.endpoint, headers=headers, params=self.params)
        print(response.json())
        
    def create_signature(self,params, secret):
        query_string = '&'.join([f"{key}={params[key]}" for key in sorted(params)])
        return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()
        




def sarCalculation(data):
    global pos
    pos='buy'
    sar = ta.SAR(data['High'], data['Low'], acceleration=0.05, maximum=0.5)
    if pos=='buy' and data['Close']>sar[-1]:
        ob = PlaceOrder()
        ob.buy()
    elif pos=='sell' and data['Close']<sar[-1]:
        ob = PlaceOrder()
        ob.sell()
    
    

columns = ['Open Time',"Open","High","Low","Close","Volume","Close Time",
           "Base asset volume","Number of trades","Taker buy volume","Taker buy base asset volume","Ignore"]
df = pd.DataFrame(data,columns=columns)

df["Open"] = df["Open"].astype(float)
df["Close"] = df["Close"].astype(float)
df["High"] = df["High"].astype(float)
df["Low"] = df["Low"].astype(float)
df["Volume"] = df["Volume"].astype(float)
df = df.set_index(pd.to_datetime(df["Open Time"],unit='ms'))

df = df.drop(['Base asset volume','Number of trades','Taker buy volume','Taker buy base asset volume','Ignore'],axis=1)


async def binance_kline(symbol,interval):
    global df
    url = f'wss://stream.binance.com:9443/ws/{symbol}@kline_{interval}'
    async with websockets.connect(url) as wb:
        while True:
            message = await wb.recv()
            message = json.loads(message)
            
            if message.get("k").get("T")== df['Close Time'].iloc[-1]:
                # update last row
                df.iloc[-1] = [
                    float(message.get('k').get('t')),
                    float(message.get('k').get('o')),
                    float(message.get('k').get('h')),
                    float(message.get('k').get('l')),
                    float(message.get('k').get('c')),
                    float(message.get('k').get('v')),
                    float(message.get('k').get('T')),   
                ]
                
            else:
                # insert a new row
                d= pd.DataFrame({
                    "Open Time":float(message.get('k').get('t')),
                    "Open":float(message.get('k').get('o')),
                    "High":float(message.get('k').get('h')),
                    "Low":float(message.get('k').get('l')),
                    "Close":float(message.get('k').get('c')),
                    "Volume":float(message.get('k').get('v')),
                   "Close Time" :float(message.get('k').get('T')),
                },index=[pd.Timestamp.now()])
                
                df = pd.concat([df,d])
        sarCalculation(df)
            
            
display(df)
await binance_kline(symbol.lower(),interval)

,Open Time,Open,High,Low,Close,Volume,Close Time
Open Time,,,,,,,
2021-12-31,1640908800000,47120.88,48548.26,45678.00,46216.93,34937.99796,1640995199999
2022-01-01,1640995200000,46216.93,47954.63,46208.37,47722.65,19604.46325,1641081599999
2022-01-02,1641081600000,47722.66,47990.00,46654.00,47286.18,18340.46040,1641167999999
2022-01-03,1641168000000,47286.18,47570.00,45696.00,46446.10,27662.07710,1641254399999
2022-01-04,1641254400000,46446.10,47557.54,45500.00,45832.01,35491.41360,1641340799999
...,...,...,...,...,...,...,...
2024-09-21,1726876800000,63201.05,63559.90,62758.00,63348.96,8375.34608,1726963199999
2024-09-22,1726963200000,63348.97,64000.00,62357.93,63578.76,14242.19892,1727049599999
2024-09-23,1727049600000,63578.76,64745.88,62538.75,63339.99,24078.05287,1727135999999


In [ ]:
!pip install Backtesting


In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from IPython.display import display
from backtesting.test import SMA, GOOG
import talib as ta

testing_df = df.drop(["Close Time","Open Time"],axis = 1)




class SmaCross(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 10)
        self.ma2 = self.I(SMA, price, 20)
        

    def next(self):
        if crossover(self.ma1, self.ma2):
            self.buy()
        elif crossover(self.ma2, self.ma1):
            self.sell()

class RsiCross(Strategy):
    def init(self):
        price = self.data.Close
        self.rsi = self.I(ta.RSI,price)
        self.overbought = 70
        self.oversell = 30
    def next(self):
        if self.rsi <= self.oversell:
            self.buy()
        elif self.rsi >= self.overbought:
            self.sell()

class MARSI(Strategy):
    def init(self):
        price = self.data.Close
        self.ma1 = self.I(SMA, price, 10)
        self.ma2 = self.I(SMA, price, 20)
        self.rsi = self.I(ta.RSI,price)
        self.overbought = 70
        self.oversell = 30
        self.sar = self.I(ta.SAR,price)
    def next(self):
        if crossover(self.ma1, self.ma2) or self.rsi <= self.oversell:
            self.buy()
        elif crossover(self.ma2, self.ma1) or self.rsi >= self.overbought:
            self.sell()

#SAR

class SARCross(Strategy):
    def init(self):
        self.high= self.data.High
        self.low = self.data.Low
        self.accerelation = 0.05
        self.maximum = 0.5
        self.sar = self.I(ta.SAR,self.high,self.low,acceleration=self.accerelation,maximum=self.maximum)
    def next(self):
        if crossover(self.data.Close,self.sar):
            self.buy()
        if crossover(self.sar,self.data.Close):
            self.sell()
        
bt = Backtest(testing_df, SARCross, commission=.002,
              exclusive_orders=True,cash=1000000)
display(testing_df)
stats = bt.run()
print(stats)
bt.plot()


In [ ]:
import hashlib
import hmac
import requests



API_KEY = "368115acbe78e27f4c7dc324b142afb0756f17045ce46fdd3ad030857b4efa91"
API_SECRET = "368115acbe78e27f4c7dc324b142afb0756f17045ce46fdd3ad030857b4efa91"
BASE_URL = 'https://testnet.binance.vision'



response = requests.get(BASE_URL + '/api/v3/time')
server_time = response.json()['serverTime']





# Endpoint for placing orders
endpoint = '/api/v3/order'



params = {
    'symbol': 'BTCUSDT',
    'side': 'BUY',
    'type': 'MARKET',
    'quantity': 0.001,
    'recvWindow': 5000,  # Time for the request to be valid in milliseconds
    'timestamp': server_time, 
}


def create_signature(params, secret):
    query_string = '&'.join([f"{key}={params[key]}" for key in sorted(params)])
    return hmac.new(secret.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()
params['signature'] = create_signature(params, API_SECRET)


headers = {
    'X-MBX-APIKEY': API_KEY
}

response = requests.post(BASE_URL + endpoint, headers=headers, params=params)
print(response.json())
